#### _Programming Exercise 2.2.2:_  Signing with a tweaked 2-of-2 MuSig key pair

In [ ]:
# Generate key pairs
privkey1, pubkey1 = generate_key_pair()
privkey2, pubkey2 = generate_key_pair()

# Create an aggregate MuSig pubkey
c_map, agg_pubkey = generate_musig_key([pubkey1, pubkey2])

# Apply challenge factors to keys
privkey1_c = privkey1 * c_map[pubkey1]
privkey2_c = privkey2 * c_map[pubkey2]
pubkey1_c = pubkey1 * c_map[pubkey1]
pubkey2_c = pubkey2 * c_map[pubkey2]

# Tweak musig public key
# Method: ECPubKey.tweak_add()
tweak = random.randrange(1, SECP256K1_ORDER)
agg_pubkey_tweaked = agg_pubkey.tweak_add(tweak)

# Nonce generation & aggregation
# Remember to negate the individual nonce values if required
# Methods: generate_schnorr_nonce(), aggregate_schnorr_nonces()
k1 = generate_schnorr_nonce()
k2 = generate_schnorr_nonce()
R1 = k1.get_pubkey()
R2 = k2.get_pubkey()
R_agg, negated = aggregate_schnorr_nonces([R1, R2])
if negated:
    k1.negate()
    k2.negate()

# Signing and signature aggregation
msg = sha256(b'msg')

# Sign individually and then aggregate partial signatures. A factor (e * tweak)
# needs to be added to the list of partial signatures
# Method: sign_musig(private_key, nonce_key, nonce_point, public_key, msg)
# Method: aggregate_musig_signatures(partial_signature_list, aggregate nonce)
# Note: The tweak multiplied by the musig_digest is added to the partial signature list.
#       This is equivalent to tweaking the private key for a single signer.
e = musig_digest(R_agg, agg_pubkey_tweaked, msg)
s1 = sign_musig(privkey1_c, k1, R_agg, agg_pubkey_tweaked, msg)
s2 = sign_musig(privkey2_c, k2, R_agg, agg_pubkey_tweaked, msg)
sig_agg = aggregate_musig_signatures([s1, s2, e * tweak], R_agg)

assert agg_pubkey_tweaked.verify_schnorr(sig_agg, msg)
print("Success!")

#### _Programming Exercise 2.2.6:_ Construct taproot output with tweaked public key

In [ ]:
# Example key pair
privkey = ECKey().set(102118636618570133408735518698955378316807974995033705330357303547139065928052)
internal_pubkey = privkey.get_pubkey()

# Example tweak
taptweak = bytes.fromhex('2a2fb476ec9962f262ff358800db0e7364287340db73e5e48db36d1c9f374e30')

# Tweak the public key
# Method: use tweak_add()
taproot_pubkey = internal_pubkey.tweak_add(taptweak)
taproot_pubkey_b = taproot_pubkey.get_bytes()

# Derive the bech32 address
# Tip: set the first byte of taproot_pubkey to 0 or 1 and then call program_to_witness(version_int, pubkey_bytes)
taproot_pubkey_v1 = bytes([taproot_pubkey_b[0] & 1]) + taproot_pubkey_b[1:]
address = program_to_witness(1, taproot_pubkey_v1)

assert address == "bcrt1pq9lku0vuddzvcte8yvt3xct0dk6cjqeq2yzqp3vwpvh2e8afqpvqqyftl09"
print("Success! Address: {}".format(address))

#### _Programming Exercise 2.2.9:_ Spend taproot output with key path

In [ ]:
# Sign transaction with tweaked private key
# Method: TaprootSignatureHash(tx, output_list, hash_type=int, input_index=int, scriptpath=bool)
sighash = TaprootSignatureHash(spending_tx, [tx.vout[0]], SIGHASH_ALL_TAPROOT, input_index=0)
tweaked_privkey = privkey.add(taptweak)
sig = tweaked_privkey.sign_schnorr(sighash)

# Add witness to transaction
spending_tx.wit.vtxinwit.append(CTxInWitness([sig]))

# Test mempool acceptance
assert node.test_transaction(spending_tx)
print("Success!")

#### _Programming Exercise 2.2.13:_ Generate segwit v1 address for a pay-to-contract public key

In [ ]:
# Generate a key pair
privkey, pubkey = generate_key_pair()

# Generate the pay-to-contract tweak
contract_bytes = "Alice pays 10 BTC to Bob".encode('utf-8')
ss = pubkey.get_bytes()
ss += sha256(contract_bytes)
t = sha256(ss)
tweak_private = ECKey().set(t, True)
tweak_point = tweak_private.get_pubkey()

# Tweak Alice's key pair with the pay-to-contract tweak
tweaked_pubkey = pubkey + tweak_point
tweaked_privkey = privkey + tweak_private

# Generate the segwit v1 address
tweaked_pubkey_data = tweaked_pubkey.get_bytes()
tweaked_pubkey_program = bytes([tweaked_pubkey_data[0] & 1]) + tweaked_pubkey_data[1:]
version = 1
address = program_to_witness(version, tweaked_pubkey_program)
print("Address encoding the segwit v1 output: ", address)